<a href="https://colab.research.google.com/github/miczkejedrzej/MNLP-project-1/blob/main/05_Predictions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
path = '/content/drive/MyDrive/France_Poland_multilanguage_shared_folder/test_unlabeled.csv'
df = pd.read_csv(path)
df.head()

Mounted at /content/drive


,item,name,description,type,category,subcategory
0,http://www.wikidata.org/entity/Q2427430,Northeast Flag Replacement,Zhang Xueliang's announcement on 29 December 1...,concept,History,historical event
1,http://www.wikidata.org/entity/Q125482,imam,Islamic leadership position,concept,philosophy and religion,religious leader
2,http://www.wikidata.org/entity/Q15789,FC Bayern Munich,"association football club in Munich, Germany",named entity,sports,sports club
3,http://www.wikidata.org/entity/Q582496,Fome Zero,program intended to eradicate hunger and extre...,named entity,politics,government agency
4,http://www.wikidata.org/entity/Q572811,Anthony Award,awards given at Bouchercon for mystery literature,named entity,Literature,literary award


# Packages

In [2]:
!pip install wikidata --quiet
!pip install spacy --quiet
!python -m spacy download en_core_web_md
!pip install fasttext --quiet


import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns

from wikidata.client import Client

from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, recall_score, precision_score
import requests
import certifi
import json
import spacy
import fasttext.util
from sklearn.preprocessing import MinMaxScaler
from collections import deque
from functools import lru_cache
from bs4 import BeautifulSoup
from PIL import Image
from io import BytesIO
import nltk
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers, models, callbacks
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.preprocessing import normalize
from google.colab import files

from collections import Counter





from tqdm.auto import tqdm
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.5/33.5 MB 32.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.4/73.4 kB 3.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [3]:
fasttext.util.download_model('en', if_exists='ignore')  # English
ft = fasttext.load_model('cc.en.300.bin')
nltk.download('stopwords')
nltk.download('punkt_tab')
porter = PorterStemmer()
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


# Features extractions

Photos

In [4]:
def extract_entity_id(url):

    """
    Function that extract the identity of an item, based on its wikidata link page

    Parameters:
    url (str): A Wikidata URL

    Returns:
    str: The entity ID extracted from the URL

    """
    return url.strip().split("/")[-1]



def get_claims(wikidata_url):

    """
    Return the claims associated with a Wikidata item.

    Parameters:
    wikidata_url (str): The URL of the Wikidata item

    Returns:
    dict: A dictionary of claims as keys, and wikidata item as values.
    """
    # Extract the id of the item
    entity_id = extract_entity_id(wikidata_url)

    # Extract information of the item
    client = Client()
    item = client.get(entity_id, load=True)

    # Extract the claims of the item
    claims = item.data.get("claims", {})

    return claims


def get_sitelinks(wikidata_url):

    """
    Return the sitelinks associated with a Wikidata item.

    Parameters:
    wikidata_url (str): The URL of the Wikidata item.

    Returns:
    dict: A dictionary with languages codes as keys, and sitelink URLs as values.
    """

    # Extract the id of the item
    entity_id = extract_entity_id(wikidata_url)

    # Extract information of the item
    client = Client()
    item = client.get(entity_id, load=True)

    # Extract the claims of the item
    sitelinks = item.data.get("sitelinks", {})

    return sitelinks

def get_en_wikipedia_url(wikidata_url):

    """
    Returns the english wikipedia page associated to a Wikidata item.

    Parameters:
    wikidata_url (str): The URL of the Wikidata item.

    Returns:
    str: A string with an english wikipedia page.
    """

    sitelinks = get_sitelinks(wikidata_url)

    # Find the english wikidepia page
    enwiki = sitelinks.get("enwiki")

    # Return its URL
    return(enwiki.get("url"))


def get_wikipedia_image(wikidata_url):

    """
    Returns the photo in the infobox, in the english wikipedia page associated to a Wikidata item.

    Parameters:
    wikidata_url (str): The URL of the Wikidata item.

    Returns:
    np.ndarray or None: The image from the Wikipedia infobox as a NumPy array, or None if there is an error in the process.
    """
    # Extract the english wikipedia page if it exists
    try:
      wikipedia_url = get_en_wikipedia_url(wikidata_url)

    except Exception:
      return None

    # Fighting against the bots protection
    headers = {'User-Agent': 'Mozilla/5.0'}
    response = requests.get(wikipedia_url, headers=headers)

    # Make sur the response has no error
    if response.status_code == 200:

        # Extract the infobox of the wikipedia page
        soup = BeautifulSoup(response.text, 'html.parser')
        infobox = soup.find('table', {'class': 'infobox'})

        if infobox:
            # Find the image
            img_tag = infobox.find('img')
            if img_tag:
                # Recreate the URL of the image
                img_url = 'https:' + img_tag['src']
                try:
                    img_response = requests.get(img_url, headers=headers)
                    img = Image.open(BytesIO(img_response.content))
                    return np.array(img)
                except:
                    return None
    # If there is an error somewhere, return None
    return None


def resize_array(image):

    """
    Resizes an image to a the size 28x28.

    Parameters:
    image (np.ndarray or None): The input image.

    Returns:
    np.ndarray or None: The resized image.
    """

    if image is not None:
      image = image.astype(np.uint8)
      img_pil = Image.fromarray(image)
      img_pil = img_pil.convert("RGB")
      img_pil = img_pil.resize((28, 28), Image.Resampling.LANCZOS)

      return np.array(img_pil)

    return None

In [5]:
# Application of the extraction of the photos

df['image'] = df['item'].apply(lambda x: get_wikipedia_image(x))

Sparql

In [6]:
WIKIDATA_SPARQL_URL = "https://query.wikidata.org/sparql"
HEADERS = {
    "Accept": "application/sparql-results+json",
    "User-Agent": "ExplorateurWikidata/0.1 (truc@truc.com)"
}


#cache avoid requesting the same item several times during the graph search
@lru_cache(maxsize=None)

def get_superclasses(qid):

    """
    Returns the superclasses (P279) of a given Wikidata item.

    Parameters:
    qid (str): The Wikidata entity identifier.

    Returns:
    tuple: A tuple of QIDs representing the superclasses of the Wikidata entity.
    """

    query = f"""
    SELECT ?superclass WHERE {{
      wd:{qid} wdt:P279 ?superclass.
    }}
    """

    response = requests.get(WIKIDATA_SPARQL_URL, params={"query": query}, headers=HEADERS)
    results = response.json()["results"]["bindings"]

    return tuple(r["superclass"]["value"].split("/")[-1] for r in results)


def get_subclass_depth_bfs(url, max_depth=20):

    """
    Return the depth of a Wikidata item in its subclass hierarchy using BFS.

    Parameters:
    url (str): The Wikidata URL of the item
    max_depth (int, optional): The maximum depth to search.

    Returns:
    int or None: The depth of the item in the subclass hierarchy
    """

    qid = extract_entity_id(url)
    visited = set()
    queue = deque([(qid, 1)])

    while queue:
        current_qid, depth = queue.popleft()
        if current_qid in visited:
            continue
        visited.add(current_qid)

        parents = get_superclasses(current_qid)
        if not parents:
            return depth
        if depth >= max_depth:
            return None
        for parent_qid in parents:
            queue.append((parent_qid, depth + 1))

    return None



def get_direct_subclasses(url):

    """
    Returns the number of direct subclasses (P279) of a given Wikidata item.

    Parameters:
    url (str): The Wikidata URL of the item.

    Returns:
    int: The number of direct subclasses of the Wikidata item.
    """
    qid = extract_entity_id(url)
    query = f"""
    SELECT ?subclass WHERE {{
      ?subclass wdt:P279 wd:{qid} .
    }}
    """
    response = requests.get(WIKIDATA_SPARQL_URL, params={"query": query}, headers=HEADERS)
    results = response.json()["results"]["bindings"]
    return len(results)


def get_instances_of(url):

    """
    Return the number of instances (P31) of a given Wikidata item.

    Parameters:
    url (str): The Wikidata URL of the item.

    Returns:
    int: The number of items that are instances of the Wikidata entity.
    """
    qid = extract_entity_id(url)
    query = f"""
    SELECT ?entity ?label WHERE {{
      ?entity wdt:P31 wd:{qid}.
      ?entity rdfs:label ?label .
      FILTER(LANG(?label) = "en")
    }}
    """
    response = requests.get(WIKIDATA_SPARQL_URL, params={"query": query}, headers=HEADERS)
    results = response.json()["results"]["bindings"]
    #return [(r["class"]["value"].split("/")[-1], r["label"]["value"]) for r in results ]
    return len(results)


def get_instances_of_up(url):

    """
    Return the number of classes that the Wikidata item is an instance of (P31).

    Parameters:
    url (str): The Wikidata URL of the item.

    Returns:
    int: The number of classes that the Wikidata item is an instance of.
    """
    qid = extract_entity_id(url)
    query = f"""
    SELECT ?class WHERE {{
        wd:{qid} wdt:P31 ?class .
      }}
    """
    response = requests.get(WIKIDATA_SPARQL_URL, params={"query": query}, headers=HEADERS)
    results = response.json()["results"]["bindings"]
    #return [(r["class"]["value"].split("/")[-1], r["label"]["value"]) for r in results ]
    return len(results)

In [7]:
# Application

tqdm.pandas()
df["subclass_depth"]= df["item"].progress_apply(get_subclass_depth_bfs)
df["subclasses"] = df["item"].apply(get_direct_subclasses)
df["instances_of"] = df["item"].apply(get_instances_of)
df["instances_of_up"] = df["item"].apply(get_instances_of_up)

  0%|          | 0/300 [00:00<?, ?it/s]

Wikipedia text

In [8]:
def get_wikipedia_extract(wikidata_url):
    """return the Wikipedia text for a given wikidata url"""

    entity_id = extract_entity_id(wikidata_url)

    client = Client()
    try:
        item = client.get(entity_id, load=True)
        # Get english wikipedia sitelink
        sitelinks = item.data.get("sitelinks", {})
        enwiki = sitelinks.get("enwiki")

        if enwiki:
            title = enwiki["title"]

            #Wikipedia API to get the text
            api_url = "https://en.wikipedia.org/w/api.php"
            headers = {'User-Agent': 'Mozilla/5.0'}
            params = {
                "action": "query",
                "prop": "extracts",
                "explaintext": True,
                "titles": title,
                "format": "json",
                "redirects": 1
            }

            res = requests.get(api_url, params=params, headers=headers).json()
            page = next(iter(res["query"]["pages"].values()))
            return page.get("extract", "No extract found.")[:1000] #1000 caracters
        else:
            return "No English Wikipedia article found for this entity."
    except requests.exceptions.HTTPError as e:
        if e.response.status_code == 404:
            print(f"Entity not found for URL: {wikidata_url}")
            return "Entity not found in Wikidata."
        else:
            raise e  # Re-raise other HTTP errors
    except Exception as e:
        # Catch any other exception that may have occurred during Wikidata entity retrieval
        print(f"An unexpected error occurred for URL: {wikidata_url}. Error: {e}")
        return "Error retrieving Wikidata entity."

In [9]:
from tqdm import tqdm
tqdm.pandas()

df['text'] = df['item'].progress_apply(get_wikipedia_extract)

100%|██████████| 300/300 [01:58<00:00,  2.53it/s]


Temporal Dimension

In [10]:
def extract_year_from_date(date):

    """
    Extracts the year from a date string.

    Parameters:
    date (str): A string representing a date

    Returns:
    str: A string representing the year extracted from the date.
    """
    return date[1:5]


def get_dates(wikidata_url):

    """
    Extract and return many dates linked to a Wikidata item.

    Parameters:
    wikidata_url (str): The URL of the Wikidata page.

    Returns:
    tuple: Contains many dates in the following order (if founded):
      - Date of birth
      - Creation date
      - Start time
      - Date of publication
      - Start time in event
    """

    creation_date = np.nan
    date_of_birth = np.nan
    date_of_publication = np.nan
    start_time = np.nan
    start_time_in_event = np.nan

    claims = get_claims(wikidata_url)

    # Extract the statement "P569" : Date of Birth
    unicode_statements = claims.get("P569", [])
    if unicode_statements:
        for statement in unicode_statements:
            mainsnak = statement.get("mainsnak", {})
            datavalue = mainsnak.get("datavalue", {})
            value = datavalue.get("value", {})
            time = value.get('time')
            if time:
                date_of_birth = extract_year_from_date(time)


    # Extract the statement "P571" : Creation Date
    unicode_statements = claims.get("P571", [])
    if unicode_statements:
        for statement in unicode_statements:
            mainsnak = statement.get("mainsnak", {})
            datavalue = mainsnak.get("datavalue", {})
            value = datavalue.get("value", {})
            time = value.get('time')
            if time:
                creation_date = extract_year_from_date(time)


    # Extract the statement "P574" : Start Time
    unicode_statements = claims.get("P574", [])
    if unicode_statements:
        for statement in unicode_statements:
            mainsnak = statement.get("mainsnak", {})
            datavalue = mainsnak.get("datavalue", {})
            value = datavalue.get("value", {})
            time = value.get('time')
            if time:
                start_time = extract_year_from_date(time)


    # Extract the statement "P577" : Date of Publication
    unicode_statements = claims.get("P577", [])
    if unicode_statements:
        for statement in unicode_statements:
            mainsnak = statement.get("mainsnak", {})
            datavalue = mainsnak.get("datavalue", {})
            value = datavalue.get("value", {})
            time = value.get('time')
            if time:
                date_of_publication = extract_year_from_date(time)


    # Extract the statement "P584" : Start Time in event
    unicode_statements = claims.get("P584", [])
    if unicode_statements:
        for statement in unicode_statements:
            mainsnak = statement.get("mainsnak", {})
            datavalue = mainsnak.get("datavalue", {})
            value = datavalue.get("value", {})
            time = value.get('time')
            if time:
                start_time_in_event = extract_year_from_date(time)

    return date_of_birth, creation_date, start_time, date_of_publication , start_time_in_event


# Due to some errors when we apply get_creation_date to the dataframe, we create a more safety function

def safe_get_dates(wikidata_url):

    """
    Add a security to the extraction of the dates linked to a Wikidata item.

    Parameters:
    wikidata_url (str): The URL of the Wikidata page.

    Returns:
    tuple: Contains many dates in the following order (if founded):
      - Date of birth
      - Creation date
      - Start time
      - Date of publication
      - Start time in event
    """
    try:
        return get_dates(wikidata_url)
    except:
        return None, None, None, None, None


# We merge the variables, in order to have only one.

def merge_variables(df, variables):

    """
    Merges multiple variables into a single variable.

    Parameters:
    df: The input dataframe containing the columns to merge.
    variables (list): A list of column names in the dataframe to be merged.

    Returns:
    pandas.Series: A new pandas Series with a single column.
    """
    return df[variables].bfill(axis=1).iloc[:, 0]

In [11]:
# Application

variables = ['date_of_birth', 'creation_date', 'start_time', 'date_of_publication', 'start_time_in_event']
df[variables] = df['item'].apply(lambda x: safe_get_dates(x)).apply(pd.Series)
df['date'] = merge_variables(df, variables)
df.drop(variables, axis=1, inplace=True)
df['date'] = pd.to_numeric(df['date'], errors='coerce').astype('Int64')

Country

In [12]:
def get_country(wikidata_url):

    """
    Extract and return many country informations linked to a Wikidata item.

    Parameters:
    wikidata_url (str): The URL of the Wikidata page.

    Returns:
    tuple: Contains many dates in the following order (if founded):
      - Country of origin
      - Country
      - Nationality
    """

    claims = get_claims(wikidata_url)
    client = Client()

    country_of_origin_id = None
    country_id = None
    nationality_id = None

    # Extract the statement "P495" : Country of origin
    unicode_statements = claims.get("P495", [])
    if unicode_statements:
        for statement in unicode_statements:
            mainsnak = statement.get("mainsnak", {})
            datavalue = mainsnak.get("datavalue", {})
            value = datavalue.get("value", {})

            country_of_origin_id = value.get("id")

            return country_of_origin_id, None, None


    # Extract the statement "P17" : Country
    unicode_statements = claims.get("P17", [])
    if unicode_statements:
        for statement in unicode_statements:
            mainsnak = statement.get("mainsnak", {})
            datavalue = mainsnak.get("datavalue", {})
            value = datavalue.get("value", {})
            country_id = value.get("id")

            return None, country_id, None


    # Extract the statement "P27" : Nationality
    unicode_statements = claims.get("P27", [])
    if unicode_statements:
        for statement in unicode_statements:
            mainsnak = statement.get("mainsnak", {})
            datavalue = mainsnak.get("datavalue", {})
            value = datavalue.get("value", {})
            nationality_id = value.get("id")

            return None, None, nationality_id

    return country_of_origin_id, country_id, nationality_id


# Due to some errors when we apply get_country to the dataframe, we create a more safety function

def safe_get_country(wikidata_url):
    """
    Add a security to the extraction of the dates linked to a Wikidata item.

    Parameters:
    wikidata_url (str): The URL of the Wikidata page.

    Returns:
    tuple: Contains many dates in the following order (if founded):
      - Country of origin
      - Country
      - Nationality
    """
    try:
        return get_country(wikidata_url)
    except:
        return None, None, None

In [13]:
# Application

df[['country_of_origin', 'country', 'nationality']] = df['item'].apply(lambda url: pd.Series(safe_get_country(url)))
df['main_country'] = merge_variables(df, ['country_of_origin', 'country', 'nationality'])
df.drop(['country_of_origin', 'country', 'nationality'], axis=1, inplace=True)

Description

In [14]:
def extract_linguistic_features(text):

    """
    Extracts features from a given text (NER => Named entity recognition, and linguistic features).


    Parameters:
    text (str): The input text.

    Returns:
    dict: A dictionary containing the following features:
        - 'descr_num_nouns': Number of nouns.
        - 'descr_num_verbs': Number of verbs.
        - 'descr_num_adjectives': Number of adjectives.
        - 'descr_has_location': 1 if a location entity is found (GPE or LOC), else 0.
        - 'descr_has_ethnic_group': 1 if an ethnic, religious, or political group entity (NORP) is found, else 0.
        - 'descr_has_event': 1 if an event entity is found, else 0.
    """
    doc = nlp(text)

    # Extract the number of nuns, verbs and adjectives in the text
    pos_counts = Counter([token.pos_ for token in doc])
    num_nouns = pos_counts.get("NOUN", 0)
    num_verbs = pos_counts.get("VERB", 0)
    num_adjectives = pos_counts.get("ADJ", 0) # adjectives can indicate a subjective tone

    # NER: named entities recognition
    ent_labels = [ent.label_ for ent in doc.ents]
    has_location = int("GPE" in ent_labels or "LOC" in ent_labels) # GPE = Geo political entity
    has_ethnic_group = int("NORP" in ent_labels)  # NORP = Nationalities, religious or political groups
    has_event = int("EVENT" in ent_labels)

    return {
        "descr_num_nouns": num_nouns,
        "descr_num_verbs": num_verbs,
        "descr_num_adjectives": num_adjectives,
        "descr_has_location": has_location,
        "descr_has_ethnic_group": has_ethnic_group,
        "descr_has_event": has_event,
    }

In [15]:
# Application

nlp = spacy.load("en_core_web_md")
df_features = df['description'].apply(extract_linguistic_features).apply(pd.Series)
df = pd.concat([df, df_features], axis=1)

Languages

In [16]:
def get_nb_lang(wikidata_url):

  """
  Return the number of languages in which a Wikidata item is defined.

  Parameters:
  wikidata_url (str): The URL of the Wikidata page.

  Returns:
  int or None: The number of languages for the item.
    """

  # Extract the id of the item
  entity_id = extract_entity_id(wikidata_url)

  #Extract informations of the item
  client = Client()
  try:
    item = client.get(entity_id, load=True)
    labels = item.data.get("labels", {})
    return len(labels.keys())
  except:
    return None

In [17]:
# Application

df['nb_lang'] = df['item'].apply(lambda x: get_nb_lang(x))

# Cleaning

In [18]:
# Type : One-hot-encoding
# We fill in by entity, as there is more 'entity' than 'concept' in the dataset
df['type_cat'] = df['type'].apply(lambda x: 0 if x == 'concept' else 1)

In [19]:
# Category : one-hot-encoding

# Definition of the categories seen in the training dataset
categories = ['politics', 'food', 'architecture', 'transportation', 'sports', 'media', 'philosophy and religion', 'literature', 'visual arts', 'fashion', 'comics and anime', 'performing arts', 'music', 'films', 'books', 'history', 'gestures and habits', 'biology', 'geography']
cols_reject = ['philosophy and religion', 'history', 'performing arts', 'gestures and habits', 'books']

# Delete the categories that has been unified in the variable 'category_combined'
categories = [category for category in categories if category not in cols_reject]

# Create the variables
for category in categories:
  name = 'category_' + category
  df[name] = df['category'].apply(lambda x: 1 if x == category else 0)

df['category_combined'] = df['category'].apply(lambda x: 1 if x not in categories else 0)

# Drop variable category
df.drop(columns=['category'], axis=1, inplace=True)

In [20]:
# Images

def fill_NaN_image(image):

  """
  If a cell is empty, it fills in with a white image, of dimension (28,28,3) ie (height, width, channels)
  """

  if image is None or (isinstance(image, float) and np.isnan(image)):
    image = np.ones((28, 28, 3), dtype=np.uint8) * 255

  return image


def clean_images(df, fill=True, drop=False):

  if fill == True:
    df['image'] = df['image'].apply(lambda x: fill_NaN_image(x))

  if drop == True:
    df = df[df['image'].notna()]

  return df


df = clean_images(df, fill=True, drop=False)

df['image'] = df['image'].apply(lambda x: json.dumps(x.tolist() if x is not None else None))

df['image'] = df['image'].apply(lambda s: np.array(json.loads(s)) if pd.notnull(s) else None)

df['image'] = df['image'].apply(lambda x: resize_array(x))


In [21]:
# Dates

current_year = int(2025)
df['date'] = df['date'].apply(lambda x: None if x > current_year else x)
df['date'] = df['date'].fillna(-1)

In [22]:
# Nb_lang

med_value = 19
df['nb_lang'] = df['nb_lang'].fillna(int(med_value))

In [23]:
# Main country

df['main_country_cat'] = df['main_country'].apply(lambda x: 0 if pd.isna(x) else 1)

In [24]:
# instances_of

med_value = 0
clipping_value = 783
df['instances_of'] = df['instances_of'].clip(upper=clipping_value)
df["instances_of"] = df["instances_of"].fillna(med_value)

In [25]:
# instances_of_up

med_value = 1
clipping_value = 5
df['instances_of_up'] = df['instances_of_up'].clip(upper=clipping_value)
df["instances_of_up"] = df["instances_of_up"].fillna(med_value)

In [26]:
# subclasses

med_value = 0
clipping_value = 56
df['subclasses'] = df['subclasses'].clip(upper=clipping_value)
df["subclasses"] = df["subclasses"].fillna(med_value)

In [27]:
# subclass_depth
med_value = 1
df["subclass_depth"] = df["subclass_depth"].fillna(med_value)

# Model 1 : Ensemble

In [28]:
from huggingface_hub import login, create_repo, upload_folder, hf_hub_download


# Repo name and model file name
repo_id = "tgarnier067/MNLP-HW1"
model_name_1 = "model_logits_features.keras"
model_name_2 = "model_logits_description.keras"
model_name_3 = "model_logits_photos.keras"
model_name_4 = "model_ensemble.keras"


# Download the model from Hugging Face
model_path_1 = hf_hub_download(repo_id=repo_id, filename=model_name_1)
model_path_2 = hf_hub_download(repo_id=repo_id, filename=model_name_2)
model_path_3 = hf_hub_download(repo_id=repo_id, filename=model_name_3)
model_path_4 = hf_hub_download(repo_id=repo_id, filename=model_name_4)


# Load the .keras model with TensorFlow
model_logits_features = tf.keras.models.load_model(model_path_1)
model_logits_description = tf.keras.models.load_model(model_path_2)
model_logits_photos = tf.keras.models.load_model(model_path_3)
model_Ensemble = tf.keras.models.load_model(model_path_4)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


model_logits_features.keras:   0%|          | 0.00/8.74M [00:00<?, ?B/s]

model_logits_description.keras:   0%|          | 0.00/9.86M [00:00<?, ?B/s]

model_logits_photos.keras:   0%|          | 0.00/517k [00:00<?, ?B/s]

model_ensemble.keras:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

In [29]:
#model_Ensemble.summary()

### Apply photos, description and features models

In [30]:
# Descritpion model

'''transforming the descriptions to the embeddings space , adding the oov information'''
def transform_description(description):
  oov_words = 0
  description = nltk.word_tokenize(description)
  description = [word for word in description if word not in stop_words]
  description = [word.lower() for word in description]
  for word in description:
    if word not in ft.words:
      oov_words += 1
  description = " ".join(description)
  description_vector = ft.get_sentence_vector(description)
  description_vector = normalize(description_vector.reshape(1, -1), norm='l2')[0]
  description_vector = np.append(description_vector, oov_words)
  return description_vector

# Extract description
texts = df['description'].to_list()

# Vectorize the texts with fasttext
vectors = np.array([transform_description(text) for text in texts])

# Apply the model
logits_model_description = model_logits_description.predict(vectors)

10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 45ms/step


In [31]:
# Features model

# Get the list of explicatives variables
list_var = ['subclass_depth', 'subclasses', 'instances_of', 'instances_of_up', 'date', 'nb_lang',
       'descr_num_nouns', 'descr_num_verbs',
       'descr_num_adjectives', 'descr_has_location', 'descr_has_ethnic_group',
       'descr_has_event', 'type_cat', 'category_architecture',
       'category_biology', 'category_comics and anime', 'category_fashion',
       'category_films', 'category_food', 'category_geography',
       'category_literature', 'category_media', 'category_music',
       'category_politics', 'category_sports', 'category_transportation',
       'category_visual arts', 'category_combined',
       'main_country_cat'
]

# Get the features
X = df[list_var]


# Selecting numerical columns from X (all of them are int64 or float64)
num_cols = X.select_dtypes(include=['int64', 'float64']).columns.tolist()

# Defining pipeline: impute Nan, then scale
num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', MinMaxScaler())
])

# Appling pipeline to numeric columns
preprocessor = ColumnTransformer([
    ('num', num_pipeline, num_cols)
])


#adjustin the data to proper format
X_processed = preprocessor.fit_transform(X)

# float cast
X_processed = X_processed.astype('float32')


logits_model_features = model_logits_features.predict(X_processed)

10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 41ms/step


In [32]:
# Photos model

df['image'] = df['image'].apply(lambda x: np.array(x))
# Normalisation des images
X = np.stack(df['image'].to_numpy()).astype('float32') / 255.0

# Apply the model
logits_model_photos = model_logits_photos.predict(X)

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step


### Apply the ensemble model

In [33]:
ensemble_vectors = np.concatenate((logits_model_description, logits_model_features), axis=1)
ensemble_vectors = np.concatenate((ensemble_vectors, logits_model_photos), axis=1)

y_pred = model_Ensemble.predict(ensemble_vectors)
y_pred_classes = np.argmax(y_pred, axis=1)

df['Ensemble_model_label'] = y_pred_classes

10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step


In [34]:
df['Ensemble_model_label']

,Ensemble_model_label
0,0
1,0
2,0
3,0
4,1
...,...
295,1
296,2
297,2
298,2


In [35]:
dict_mapping = {'cultural exclusive' : 0,
                'cultural representative' : 1,
                'cultural agnostic' : 2}

inverted_dict_mapping = {v: k for k, v in dict_mapping.items()}

df['Ensemble_model_label'] = df['Ensemble_model_label'].map(inverted_dict_mapping)

# Transformers

In [36]:
!pip install datasets --quiet

from datasets import Dataset


df["idx"] = range(len(df))

# Creation of the dictionnary datasets
dictionnary = df[['idx','name','description']].to_dict(orient="list")
dataset = Dataset.from_dict(dictionnary)

# Visualisation
dataset

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 17.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cu

Dataset({
    features: ['idx', 'name', 'description'],
    num_rows: 300
})

In [37]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# Remplacer par l'ID de ton modèle
model_name = "tgarnier067/MNLP-HW1"

# Charger le tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Charger le modèle pour la classification de séquences
model = AutoModelForSequenceClassification.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/723 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [38]:
def tokenize_function(examples):
    text_inputs = [f"{name}: {desc}" for name, desc in zip(examples["name"], examples["description"])]
    return tokenizer(text_inputs, padding=True, truncation=True)

In [39]:
tokenized_dataset = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

In [40]:
def predict_culture(text, model, tokenizer, device, max_length=128):
    model.eval()
    encoding = tokenizer(text, return_tensors='pt', max_length=max_length, padding='max_length', truncation=True)
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)
    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        _, preds = torch.max(outputs.logits, dim=1)
    return (
        "cultural exclusive" if preds.item() == 0
        else "cultural representative" if preds.item() == 1
        else "cultural agnostic"
    )

In [41]:
def build_input_text(name, description):
    return f"the name of the entity:{name}. The description of the entity:{description}"

df["input_text"] = df.apply(lambda row: build_input_text(row["name"], row["description"]), axis=1)

device = "cuda" if torch.cuda.is_available() else "cpu"

df["Transformer_model_label"] = df["input_text"].apply(lambda text: predict_culture(text, model, tokenizer, device))


In [42]:
df["Transformer_model_label"]

,Transformer_model_label
0,cultural exclusive
1,cultural representative
2,cultural exclusive
3,cultural exclusive
4,cultural representative
...,...
295,cultural agnostic
296,cultural agnostic
297,cultural agnostic
298,cultural agnostic


# Final tables

In [60]:
ensemble_df = df[['item', 'name', 'description', 'type', 'subcategory', 'Ensemble_model_label']].rename(columns={'Ensemble_model_label': 'label'})

transformer_df = df[['item', 'name', 'description', 'type', 'subcategory', 'Transformer_model_label']].rename(columns={'Transformer_model_label' : 'label'})

In [62]:
ensemble_df.head()

,item,name,description,type,subcategory,label
0,http://www.wikidata.org/entity/Q2427430,Northeast Flag Replacement,Zhang Xueliang's announcement on 29 December 1...,concept,historical event,cultural exclusive
1,http://www.wikidata.org/entity/Q125482,imam,Islamic leadership position,concept,religious leader,cultural exclusive
2,http://www.wikidata.org/entity/Q15789,FC Bayern Munich,"association football club in Munich, Germany",named entity,sports club,cultural exclusive
3,http://www.wikidata.org/entity/Q582496,Fome Zero,program intended to eradicate hunger and extre...,named entity,government agency,cultural exclusive
4,http://www.wikidata.org/entity/Q572811,Anthony Award,awards given at Bouchercon for mystery literature,named entity,literary award,cultural representative


In [61]:
transformer_df.head()

,item,name,description,type,subcategory,label
0,http://www.wikidata.org/entity/Q2427430,Northeast Flag Replacement,Zhang Xueliang's announcement on 29 December 1...,concept,historical event,cultural exclusive
1,http://www.wikidata.org/entity/Q125482,imam,Islamic leadership position,concept,religious leader,cultural representative
2,http://www.wikidata.org/entity/Q15789,FC Bayern Munich,"association football club in Munich, Germany",named entity,sports club,cultural exclusive
3,http://www.wikidata.org/entity/Q582496,Fome Zero,program intended to eradicate hunger and extre...,named entity,government agency,cultural exclusive
4,http://www.wikidata.org/entity/Q572811,Anthony Award,awards given at Bouchercon for mystery literature,named entity,literary award,cultural representative


Exportation

In [44]:
ensemble_df.to_csv('France_Poland_multilanguage_Non_transformer_model.csv')
transformer_df.to_csv('France_Poland_multilanguage_transformer_model.csv')

from google.colab import files

files.download('France_Poland_multilanguage_Non_transformer_model.csv')
files.download('France_Poland_multilanguage_transformer_model.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>